In [101]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [102]:
train_ds = tf.keras.utils.image_dataset_from_directory("./data/akshar",
                                                       subset = "training",
                                                       validation_split=0.2,
                                                       shuffle=True,
                                                       image_size = (28, 28),
                                                       color_mode="grayscale",
                                                      seed = 777)

val_ds = tf.keras.utils.image_dataset_from_directory("./data/akshar",
                                                         validation_split = 0.2,
                                                     image_size = (28, 28),
                                                     shuffle=True,
                                                     color_mode = "grayscale",
                                                     subset="validation",
  seed=777)
class_names = train_ds.class_names

Found 3369 files belonging to 34 classes.
Using 2696 files for training.
Found 3369 files belonging to 34 classes.
Using 673 files for validation.


In [103]:
num_classes = len(train_ds.class_names)
print(train_ds.class_names)
textfile = open("class_names.txt", "w")
for element in class_names:
    textfile.write(element + "\n")
textfile.close()

['ALA', 'ANA', 'B', 'BHA', 'CH', 'CHH', 'D', 'DA', 'DH', 'DHA', 'F', 'G', 'GH', 'GNA', 'H', 'J', 'JH', 'K', 'KH', 'KSH', 'L', 'M', 'N', 'P', 'R', 'S', 'SH', 'SHH', 'T', 'TA', 'TH', 'THA', 'V', 'Y']


In [104]:


AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)



# need to add a layer that converts everything to grayscale!
normalization_layer = tf.keras.layers.Rescaling(1./255)
normalized_train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
normalized_val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))



Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [105]:
import tensorflow_hub as hub
mobilenet_v2 = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"
feature_extractor_model = mobilenet_v2
feature_extractor_layer = hub.KerasLayer(
    feature_extractor_model,
    input_shape=(224, 224, 3),
    trainable=False)

In [106]:


def create_simple_model():
    x = tf.keras.Input((224, 224,3))
    features = feature_extractor_layer(x)
    d1 = tf.keras.layers.Dense(num_classes, activation = "softmax")(features)
    return tf.keras.Model(x, d1)

def create_from_scratch():
    inputs = tf.keras.Input((28, 28, 1))
    x = tf.keras.layers.Conv2D(4, 2)(inputs)
    x = tf.keras.layers.MaxPooling2D()(x)
    x = tf.keras.layers.Conv2D(2, 1)(x)
    x = tf.keras.layers.MaxPooling2D()(x)
    x = tf.keras.layers.Flatten()(x)
    out = tf.keras.layers.Dense(num_classes, activation = "softmax")(x)
    return tf.keras.Model(inputs, out)

#model = create_simple_model()
model = create_from_scratch()
model.summary()


Model: "model_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_18 (InputLayer)       [(None, 28, 28, 1)]       0         
                                                                 
 conv2d_19 (Conv2D)          (None, 27, 27, 4)         20        
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 13, 13, 4)        0         
 g2D)                                                            
                                                                 
 conv2d_20 (Conv2D)          (None, 13, 13, 2)         10        
                                                                 
 max_pooling2d_14 (MaxPoolin  (None, 6, 6, 2)          0         
 g2D)                                                            
                                                                 
 flatten_12 (Flatten)        (None, 72)                0  

In [107]:
import datetime 
model.compile(
  optimizer=tf.keras.optimizers.Adam(),
  loss=tf.keras.losses.SparseCategoricalCrossentropy(),
  metrics=['acc'])

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=log_dir,
    histogram_freq=1)

In [ ]:
NUM_EPOCHS = 100

history = model.fit(train_ds,
                    validation_data = val_ds,
                    epochs=NUM_EPOCHS,
                    callbacks=tensorboard_callback)

Epoch 1/100
85/85 [==============================] - 0s 4ms/step - loss: 3.4705 - acc: 0.3646 - val_loss: 3.3174 - val_acc: 0.3848
Epoch 2/100
85/85 [==============================] - 0s 4ms/step - loss: 3.2298 - acc: 0.3861 - val_loss: 3.2071 - val_acc: 0.3967
Epoch 3/100
85/85 [==============================] - 0s 4ms/step - loss: 2.9596 - acc: 0.4203 - val_loss: 2.8248 - val_acc: 0.4294
Epoch 4/100
85/85 [==============================] - 0s 4ms/step - loss: 2.7861 - acc: 0.4340 - val_loss: 2.6646 - val_acc: 0.4443
Epoch 5/100
85/85 [==============================] - 0s 4ms/step - loss: 2.6841 - acc: 0.4347 - val_loss: 2.6010 - val_acc: 0.4086
Epoch 6/100
85/85 [==============================] - 0s 4ms/step - loss: 2.4932 - acc: 0.4488 - val_loss: 2.3947 - val_acc: 0.4502
Epoch 7/100
85/85 [==============================] - 0s 4ms/step - loss: 2.3224 - acc: 0.4733 - val_loss: 2.3683 - val_acc: 0.4502
Epoch 8/100
85/85 [==============================] - 0s 4ms/step - loss: 2.1608 - a

In [ ]:
validation_preds = model.predict(val_ds)


In [ ]:
validation_preds[0]

In [ ]:
import matplotlib.pyplot as plt

val_ds = tf.keras.utils.image_dataset_from_directory("./data/akshar",
                                                         validation_split = 0.2,
                                                     image_size = (28, 28),
                                                     shuffle=True,
                                                     color_mode = "grayscale",
                                                     subset="validation",
  seed=777)
image_batch, label_batch = val_ds.as_numpy_iterator().next()
vp = model.predict_on_batch(image_batch)


In [ ]:
vp = tf.argmax(input=vp, axis=1)

In [ ]:

plt.figure(figsize=(10, 10))
for i in range(9):
    print(class_names[label_batch[i]])
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(image_batch[i].astype("uint8"))
    plt.title(class_names[vp[i]])
    plt.axis("off")

In [ ]:
vp

In [ ]:
model.save("models/scratch_guju")